## Data Exploration and Feature Engineering

In [59]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.window import Window
from snowflake.ml.modeling.preprocessing import *
from snowflake.ml.modeling.impute import *

import sys
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Create Snowpark Session

In [36]:
with open('creds.json') as f:
    connection_parameters = json.load(f)

In [37]:
session = Session.builder.configs(connection_parameters).create()
print(f"Current Database and schema: {session.get_fully_qualified_current_schema()}")
print(f"Current Warehouse: {session.get_current_warehouse()}")

Current Database and schema: "CC_DB"."PUBLIC"
Current Warehouse: "CC_WH"


Snowpark DataFrames vs. Pandas DataFrames

In [38]:
# Creating a Pandas DataFrame
pandas_df = pd.read_csv('RawData/application_record.csv.zip')
print(type(pandas_df))

<class 'pandas.core.frame.DataFrame'>


In [39]:
# Creating a Snowpark DataFrame
snowpark_df = session.table('APPLICATION_RECORD')
print(type(snowpark_df))

<class 'snowflake.snowpark.table.Table'>


In [40]:
# Compare size
print('Size in MB of Pandas DataFrame in Memory:\n', np.round(sys.getsizeof(pandas_df) / (1024.0**2), 2))
print('Size in MB of Snowpark DataFrame in Memory:\n', np.round(sys.getsizeof(snowpark_df) / (1024.0**2), 2))

Size in MB of Pandas DataFrame in Memory:
 251.15
Size in MB of Snowpark DataFrame in Memory:
 0.0


The only thing stored in a Snowpark DataFrame is the SQL needed to return data.
The snowpark Dataframe can also be converted to pandas Dataframe. This will pull the data from Snowflake into the Python Environment memory

In [41]:
snowpark_df.queries

{'queries': ['SELECT  *  FROM APPLICATION_RECORD'], 'post_actions': []}

In [42]:
pandas_df2 = snowpark_df.to_pandas()

In [43]:
pandas_df2.shape, pandas_df.shape

((438557, 18), (438557, 18))

In [44]:
# Showing Snowpark df
snowpark_df.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"     |"CODE_GENDER"  |"FLAG_OWN_CAR"  |"FLAG_OWN_REALTY"  |"CNT_CHILDREN"  |"AMT_INCOME_TOTAL"  |"NAME_INCOME_TYPE"    |"NAME_EDUCATION_TYPE"          |"NAME_FAMILY_STATUS"  |"NAME_HOUSING_TYPE"  |"DAYS_BIRTH"  |"DAYS_EMPLOYED"  |"FLAG_MOBIL"  |"FLAG_WORK_PHONE"  |"FLAG_PHONE"  |"FLAG_EMAIL"  |"OCCUPATION_TYPE"  |"CNT_FAM_MEMBERS"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [45]:
# collects first 5 rows and displays as pandas-dataframe
snowpark_df.limit(5).to_pandas() 

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


### Transformations

In [46]:
# Select Specific Columns
snowpark_df = snowpark_df.select('CODE_GENDER','NAME_INCOME_TYPE','DAYS_BIRTH',)
# pandas-like selection
snowpark_df = snowpark_df[['CODE_GENDER','NAME_INCOME_TYPE','DAYS_BIRTH']] 
snowpark_df.show()

-------------------------------------------------------
|"CODE_GENDER"  |"NAME_INCOME_TYPE"    |"DAYS_BIRTH"  |
-------------------------------------------------------
|M              |Working               |-12005        |
|M              |Working               |-12005        |
|M              |Working               |-21474        |
|F              |Commercial associate  |-19110        |
|F              |Commercial associate  |-19110        |
|F              |Commercial associate  |-19110        |
|F              |Commercial associate  |-19110        |
|F              |Pensioner             |-22464        |
|F              |Pensioner             |-22464        |
|F              |Pensioner             |-22464        |
-------------------------------------------------------



In [47]:
# adding a neew column, AGE, that calculates the number of years that DAYS_BIRTH is.
snowpark_df = snowpark_df.withColumn('AGE', F.floor(F.abs(F.col('DAYS_BIRTH')) / 365))
snowpark_df.show()

---------------------------------------------------------------
|"CODE_GENDER"  |"NAME_INCOME_TYPE"    |"DAYS_BIRTH"  |"AGE"  |
---------------------------------------------------------------
|M              |Working               |-12005        |32     |
|M              |Working               |-12005        |32     |
|M              |Working               |-21474        |58     |
|F              |Commercial associate  |-19110        |52     |
|F              |Commercial associate  |-19110        |52     |
|F              |Commercial associate  |-19110        |52     |
|F              |Commercial associate  |-19110        |52     |
|F              |Pensioner             |-22464        |61     |
|F              |Pensioner             |-22464        |61     |
|F              |Pensioner             |-22464        |61     |
---------------------------------------------------------------



In [48]:
# Drop a column
snowpark_df = snowpark_df.drop('DAYS_BIRTH')
snowpark_df.show()

------------------------------------------------
|"CODE_GENDER"  |"NAME_INCOME_TYPE"    |"AGE"  |
------------------------------------------------
|M              |Working               |32     |
|M              |Working               |32     |
|M              |Working               |58     |
|F              |Commercial associate  |52     |
|F              |Commercial associate  |52     |
|F              |Commercial associate  |52     |
|F              |Commercial associate  |52     |
|F              |Pensioner             |61     |
|F              |Pensioner             |61     |
|F              |Pensioner             |61     |
------------------------------------------------



In [49]:
# Filter data
snowpark_df = snowpark_df.filter(F.col('NAME_INCOME_TYPE').in_(['Pensioner','Student']))
snowpark_df.show()

----------------------------------------------
|"CODE_GENDER"  |"NAME_INCOME_TYPE"  |"AGE"  |
----------------------------------------------
|F              |Pensioner           |61     |
|F              |Pensioner           |61     |
|F              |Pensioner           |61     |
|F              |Pensioner           |55     |
|F              |Pensioner           |61     |
|F              |Pensioner           |61     |
|F              |Pensioner           |61     |
|F              |Pensioner           |61     |
|F              |Pensioner           |61     |
|F              |Pensioner           |61     |
----------------------------------------------



In [50]:
# Aggregate data by using group_by method in combination with the agg method.
snowpark_df = snowpark_df.group_by(['CODE_GENDER','NAME_INCOME_TYPE']).agg([F.avg('AGE').as_('AVG_AGE')])
snowpark_df.show()

--------------------------------------------------
|"CODE_GENDER"  |"NAME_INCOME_TYPE"  |"AVG_AGE"  |
--------------------------------------------------
|F              |Pensioner           |59.188624  |
|M              |Pensioner           |57.685482  |
|F              |Student             |46.090909  |
|M              |Student             |27.166667  |
--------------------------------------------------



In [51]:
# Sort data by using the sort method.
snowpark_df = snowpark_df.sort(F.col('AVG_AGE').desc())
snowpark_df.show()

--------------------------------------------------
|"CODE_GENDER"  |"NAME_INCOME_TYPE"  |"AVG_AGE"  |
--------------------------------------------------
|F              |Pensioner           |59.188624  |
|M              |Pensioner           |57.685482  |
|F              |Student             |46.090909  |
|M              |Student             |27.166667  |
--------------------------------------------------



### Data Analysis

In this section we will use API Snowpark to do some basic analysis of our data.


In [52]:
# Creating a Snowpark DataFrame
snowpark_df = session.table('APPLICATION_RECORD')

In [53]:
# Number of rows in dataset
snowpark_df.count()

438557

In [54]:
# drop duplicates based on ID
snowpark_df = snowpark_df.drop_duplicates('ID')
snowpark_df.count()

438510

In [55]:
# Duplicated rows are only filtered and we can see the logic for it by examining the SQL for the DataFrame, using ['queries'][0] will return the first SQL statement for the DataFrame

print(snowpark_df.queries['queries'][0])

SELECT "ID", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "DAYS_BIRTH", "DAYS_EMPLOYED", "FLAG_MOBIL", "FLAG_WORK_PHONE", "FLAG_PHONE", "FLAG_EMAIL", "OCCUPATION_TYPE", "CNT_FAM_MEMBERS" FROM ( SELECT "ID", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "DAYS_BIRTH", "DAYS_EMPLOYED", "FLAG_MOBIL", "FLAG_WORK_PHONE", "FLAG_PHONE", "FLAG_EMAIL", "OCCUPATION_TYPE", "CNT_FAM_MEMBERS", row_number() OVER (PARTITION BY "ID"  ORDER BY "ID" ASC NULLS FIRST ) AS "QUVKSHA8E9" FROM APPLICATION_RECORD) WHERE ("QUVKSHA8E9" = 1)


In [56]:
# Calculating various statistics per column
snowpark_df.describe().show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUMMARY"  |"ID"               |"CODE_GENDER"  |"FLAG_OWN_CAR"  |"FLAG_OWN_REALTY"  |"CNT_CHILDREN"      |"AMT_INCOME_TOTAL"  |"NAME_INCOME_TYPE"    |"NAME_EDUCATION_TYPE"          |"NAME_FAMILY_STATUS"  |"NAME_HOUSING_TYPE"  |"DAYS_BIRTH"       |"DAYS_EMPLOYED"     |"FLAG_MOBIL"  |"FLAG_WORK_PHONE"   |"FLAG_PHONE"         |"FLAG_EMAIL"        |"OCCUPATION_TYPE"     |"CNT_FAM_MEMBERS"   |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [57]:
# Average Income per Income Type and Gender
analysis_df = snowpark_df.group_by(['NAME_INCOME_TYPE','CODE_GENDER']).agg([F.mean('AMT_INCOME_TOTAL').as_('AVG_INCOME')])
analysis_df = analysis_df.sort('NAME_INCOME_TYPE', F.col('AVG_INCOME').desc())
analysis_df.show()

-------------------------------------------------------------
|"NAME_INCOME_TYPE"    |"CODE_GENDER"  |"AVG_INCOME"        |
-------------------------------------------------------------
|Commercial associate  |M              |249208.08642289176  |
|Commercial associate  |F              |206579.17463258584  |
|Pensioner             |M              |169049.77416737832  |
|Pensioner             |F              |150729.61255448588  |
|State servant         |M              |237034.15414285715  |
|State servant         |F              |186152.9842904419   |
|Student               |F              |165272.72727272726  |
|Student               |M              |149250.0            |
|Working               |M              |202170.82427397132  |
|Working               |F              |168679.56899413437  |
-------------------------------------------------------------



In [62]:
# We can use the SimpleImputer in snowflake.ml.preprocessing to replace missing values with the most frequent.

my_imputer = SimpleImputer(input_cols=['OCCUPATION_TYPE'], output_cols=['OCCUPATION_TYPE'] ,strategy='most_frequent')
my_imputer.fit(snowpark_df)
snowpark_df = my_imputer.transform(snowpark_df)
snowpark_df.describe().show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUMMARY"  |"OCCUPATION_TYPE"     |"ID"               |"CODE_GENDER"  |"FLAG_OWN_CAR"  |"FLAG_OWN_REALTY"  |"CNT_CHILDREN"      |"AMT_INCOME_TOTAL"  |"NAME_INCOME_TYPE"    |"NAME_EDUCATION_TYPE"          |"NAME_FAMILY_STATUS"  |"NAME_HOUSING_TYPE"  |"DAYS_BIRTH"       |"DAYS_EMPLOYED"     |"FLAG_MOBIL"  |"FLAG_WORK_PHONE"   |"FLAG_PHONE"         |"FLAG_EMAIL"        |"CNT_FAM_MEMBERS"   |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [63]:
# The missing value handling logic is converted in SQL to iff("OCCUPATION_TYPE" IS NULL, 'Laborers', "OCCUPATION_TYPE") AS "OCCUPATION_TYPE"
print(snowpark_df.queries['queries'][0])

SELECT iff("OCCUPATION_TYPE" IS NULL, 'Laborers', "OCCUPATION_TYPE") AS "OCCUPATION_TYPE", "ID", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "DAYS_BIRTH", "DAYS_EMPLOYED", "FLAG_MOBIL", "FLAG_WORK_PHONE", "FLAG_PHONE", "FLAG_EMAIL", "CNT_FAM_MEMBERS" FROM ( SELECT "ID", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "DAYS_BIRTH", "DAYS_EMPLOYED", "FLAG_MOBIL", "FLAG_WORK_PHONE", "FLAG_PHONE", "FLAG_EMAIL", "CNT_FAM_MEMBERS", "OCCUPATION_TYPE" AS "OCCUPATION_TYPE" FROM ( SELECT iff("OCCUPATION_TYPE" IS NULL, 'Laborers', "OCCUPATION_TYPE") AS "OCCUPATION_TYPE", "ID", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "DA

In [65]:
snowpark_df.schema

StructType([StructField('OCCUPATION_TYPE', StringType(), nullable=True), StructField('ID', LongType(), nullable=True), StructField('CODE_GENDER', StringType(), nullable=True), StructField('FLAG_OWN_CAR', StringType(), nullable=True), StructField('FLAG_OWN_REALTY', StringType(), nullable=True), StructField('CNT_CHILDREN', LongType(), nullable=True), StructField('AMT_INCOME_TOTAL', DoubleType(), nullable=True), StructField('NAME_INCOME_TYPE', StringType(), nullable=True), StructField('NAME_EDUCATION_TYPE', StringType(), nullable=True), StructField('NAME_FAMILY_STATUS', StringType(), nullable=True), StructField('NAME_HOUSING_TYPE', StringType(), nullable=True), StructField('DAYS_BIRTH', LongType(), nullable=True), StructField('DAYS_EMPLOYED', LongType(), nullable=True), StructField('FLAG_MOBIL', LongType(), nullable=True), StructField('FLAG_WORK_PHONE', LongType(), nullable=True), StructField('FLAG_PHONE', LongType(), nullable=True), StructField('FLAG_EMAIL', LongType(), nullable=True), S

In [66]:
#We can iterate through the schema to get the columns of specific data types

# Get all categorical columns
categorical_types = [T.StringType]
categorical_columns = [c.name for c in snowpark_df.schema.fields if type(c.datatype) in categorical_types]
categorical_columns

['OCCUPATION_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE']

In [67]:
#We can then use those to loop through and get the number of unique values

# Number of unique values per categorical column
unique_values = []
for column in categorical_columns:
    unique_values.append([column, snowpark_df.select(column).distinct().count()])
pd.DataFrame(unique_values, columns=['COLUMN_NAME','NUM_UNIQUE_VALUES'])

,COLUMN_NAME,NUM_UNIQUE_VALUES
0,OCCUPATION_TYPE,18
1,CODE_GENDER,2
2,FLAG_OWN_CAR,2
3,FLAG_OWN_REALTY,2
4,NAME_INCOME_TYPE,5
5,NAME_EDUCATION_TYPE,5
6,NAME_FAMILY_STATUS,5
7,NAME_HOUSING_TYPE,6


### Persist Transformations

If we want to save the changes we can either save it as a table, meaning the SQL generated by the DataFrame is executed and the result is stored in a table or as a view where the DataFrame SQL will be the definition of the view.
save_as_table saves the result in a table, if mode='overwrite' then it will also replace the data that is in it.

In [68]:
snowpark_df.write.save_as_table(table_name='FIRST_ANALYSIS', mode='overwrite')
session.table('FIRST_ANALYSIS').show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"OCCUPATION_TYPE"  |"ID"     |"CODE_GENDER"  |"FLAG_OWN_CAR"  |"FLAG_OWN_REALTY"  |"CNT_CHILDREN"  |"AMT_INCOME_TOTAL"  |"NAME_INCOME_TYPE"    |"NAME_EDUCATION_TYPE"          |"NAME_FAMILY_STATUS"  |"NAME_HOUSING_TYPE"  |"DAYS_BIRTH"  |"DAYS_EMPLOYED"  |"FLAG_MOBIL"  |"FLAG_WORK_PHONE"  |"FLAG_PHONE"  |"FLAG_EMAIL"  |"CNT_FAM_MEMBERS"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [69]:
session.close()

Answering some questions about the data

In [96]:
session = Session.builder.configs(connection_parameters).create()
print(f"Current Database and schema: {session.get_fully_qualified_current_schema()}")
print(f"Current Warehouse: {session.get_current_warehouse()}")

Current Database and schema: "CC_DB"."PUBLIC"
Current Warehouse: "CC_WH"


In [97]:
snowpark_df = session.table('APPLICATION_RECORD')
snowpark_df.limit(10).to_pandas()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
5,5008810,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
6,5008811,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
7,5008812,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,None,1.0
8,5008813,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,None,1.0
9,5008814,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,None,1.0


In [98]:
snowpark_df.columns

['ID',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'FLAG_MOBIL',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS']

In [99]:
# How many variables (columns) does the dataset have?
len(snowpark_df.columns)

18

In [100]:
# How many rows does the dataset have?
snowpark_df.count()

438557

In [103]:
# What is the average income?
snowpark_df.agg(F.avg('AMT_INCOME_TOTAL')).show()
snowpark_df.select(F.avg('AMT_INCOME_TOTAL').as_('AVERAGE_INCOME')).show()

---------------------------
|"AVG(AMT_INCOME_TOTAL)"  |
---------------------------
|187524.2860095039        |
---------------------------

---------------------
|"AVERAGE_INCOME"   |
---------------------
|187524.2860095039  |
---------------------



In [104]:
# How many people are Single?
snowpark_df.filter(F.col('NAME_FAMILY_STATUS') == 'Single / not married').count()

55271

In [ ]:
# Create a new feature containing the years of employment
snowpark_df = snowpark_df.withColumn('YEARS_EMPLOYED', F.floor(F.abs(F.col('DAYS_EMPLOYED')) / 365))
snowpark_df.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"     |"CODE_GENDER"  |"FLAG_OWN_CAR"  |"FLAG_OWN_REALTY"  |"CNT_CHILDREN"  |"AMT_INCOME_TOTAL"  |"NAME_INCOME_TYPE"    |"NAME_EDUCATION_TYPE"          |"NAME_FAMILY_STATUS"  |"NAME_HOUSING_TYPE"  |"DAYS_BIRTH"  |"DAYS_EMPLOYED"  |"FLAG_MOBIL"  |"FLAG_WORK_PHONE"  |"FLAG_PHONE"  |"FLAG_EMAIL"  |"OCCUPATION_TYPE"  |"CNT_FAM_MEMBERS"  |"YEARS_EMPLOYED"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [93]:
# Drop the variable DAYS_EMPLOYED
snowpark_df = snowpark_df.drop('DAYS_EMPLOYED')
snowpark_df.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"     |"CODE_GENDER"  |"FLAG_OWN_CAR"  |"FLAG_OWN_REALTY"  |"CNT_CHILDREN"  |"AMT_INCOME_TOTAL"  |"NAME_INCOME_TYPE"    |"NAME_EDUCATION_TYPE"          |"NAME_FAMILY_STATUS"  |"NAME_HOUSING_TYPE"  |"DAYS_BIRTH"  |"FLAG_MOBIL"  |"FLAG_WORK_PHONE"  |"FLAG_PHONE"  |"FLAG_EMAIL"  |"OCCUPATION_TYPE"  |"CNT_FAM_MEMBERS"  |"YEARS_EMPLOYED"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [94]:
session.close()